In [3]:
import os
import re
import pandas as pd
import numpy as np
import nltk
import pickle

%load_ext autoreload
%autoreload 2

import parent_modules
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import pairwise_distances

from definitions import *
from helpers import MovieLensRatingsDataset
from baseline_cf import *
movielens_dataset = MovieLensRatingsDataset()


In [2]:
# Jaccard Similarity
movielens_dataset.jaccard_similarity.head()


NameError: name 'movielens_dataset' is not defined

In [5]:
# Pearson Similarity
movielens_dataset.pearson_similarity.head()

movieId,1,2,3,5,6,7,9,10,11,12,...,159093,164179,166528,168250,168252,174055,176371,177765,179819,187593
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.178983,0.136128,0.153725,0.122948,0.099621,0.112117,0.185387,0.144952,0.149678,...,-0.016475,0.079961,0.028192,0.018223,0.057158,0.061225,0.057341,-0.010076,0.051466,0.014426
2,0.178983,1.000000,0.173598,0.179390,0.101651,0.104321,-0.023199,0.258169,0.124763,0.082600,...,0.009875,0.129452,0.145067,0.022873,0.219752,0.048388,0.154305,-0.033434,0.110026,0.117877
3,0.136128,0.173598,1.000000,0.359519,0.122400,0.373282,0.254815,0.129034,0.164854,0.202242,...,-0.055394,-0.008482,-0.016528,-0.043928,-0.004138,0.045740,0.033977,-0.050395,0.024357,-0.053386
5,0.153725,0.179390,0.359519,1.000000,0.135922,0.458714,0.216356,0.144774,0.203560,0.117635,...,0.020997,0.006304,0.002984,-0.003354,-0.022626,-0.003864,0.002731,-0.045835,-0.001546,-0.048556
6,0.122948,0.101651,0.122400,0.135922,1.000000,0.100664,0.138454,0.254995,0.159150,0.131322,...,0.017029,0.127574,0.083804,0.029891,0.106940,0.073934,0.124326,-0.082322,0.000549,0.028282


In [6]:
# Cosine Similarity
movielens_dataset.cosine_similarity.head()

movieId,1,2,3,5,6,7,9,10,11,12,...,159093,164179,166528,168250,168252,174055,176371,177765,179819,187593
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.446971,0.305854,0.302137,0.396841,0.271328,0.183813,0.463520,0.358906,0.225869,...,0.095783,0.213547,0.184635,0.130367,0.190300,0.152449,0.169952,0.090992,0.137871,0.114891
2,0.446971,1.000000,0.301151,0.294109,0.317119,0.237974,0.056544,0.452402,0.295874,0.157833,...,0.090493,0.226755,0.246286,0.108297,0.298076,0.122652,0.228011,0.046223,0.169823,0.179888
3,0.305854,0.301151,1.000000,0.418275,0.253466,0.433397,0.287134,0.269804,0.268005,0.244136,...,0.000000,0.066637,0.063582,0.015093,0.067422,0.094188,0.092896,0.000000,0.070658,0.000000
5,0.302137,0.294109,0.418275,1.000000,0.253221,0.506340,0.248879,0.269911,0.293094,0.161519,...,0.067381,0.074178,0.075035,0.048315,0.044274,0.043489,0.058785,0.000000,0.042411,0.000000
6,0.396841,0.317119,0.253466,0.253221,1.000000,0.229346,0.193095,0.441042,0.316268,0.197054,...,0.093126,0.221449,0.190364,0.110655,0.199210,0.141733,0.199744,0.000000,0.072779,0.100128


In [7]:
# Hamming Similarity
movielens_dataset.hamming_similarity.head()

movieId,1,2,3,5,6,7,9,10,11,12,...,159093,164179,166528,168250,168252,174055,176371,177765,179819,187593
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.490909,0.524675,0.532468,0.488312,0.519481,0.537662,0.462338,0.503896,0.537662,...,0.537662,0.532468,0.529870,0.537662,0.532468,0.542857,0.535065,0.532468,0.537662,0.532468
2,0.490909,1.000000,0.703896,0.701299,0.615584,0.690909,0.714286,0.631169,0.651948,0.719481,...,0.714286,0.709091,0.703896,0.714286,0.716883,0.716883,0.719481,0.714286,0.719481,0.724675
3,0.524675,0.703896,1.000000,0.854545,0.711688,0.838961,0.880519,0.667532,0.758442,0.880519,...,0.859740,0.841558,0.833766,0.857143,0.846753,0.864935,0.857143,0.862338,0.862338,0.862338
5,0.532468,0.701299,0.854545,1.000000,0.724675,0.867532,0.893506,0.680519,0.781818,0.883117,...,0.880519,0.857143,0.849351,0.875325,0.859740,0.880519,0.872727,0.880519,0.880519,0.880519
6,0.488312,0.615584,0.711688,0.724675,1.000000,0.719481,0.753247,0.649351,0.685714,0.745455,...,0.740260,0.732468,0.729870,0.737662,0.740260,0.742857,0.740260,0.735065,0.740260,0.742857


In [ ]:
# Calculate Baseline Df
# Read the do
baseline_cf_matrix = load_baseline_cf(
    load_from_pickle=True,
    drop
    df=movielens_dataset.full_df)
baseline_cf_matrix.shape

Estimating biases using als...


In [23]:
baseline_cf_values = []

for pred in train_pred + test_pred:
    baseline_cf_values.append({
        "movieId": pred.iid,
        "userId": pred.uid,
        "rating": pred.est
    })
baseline_cf_df = pd.DataFrame(baseline_cf_values)
# pivot ratings into movie features
baseline_cf_df_pivot = baseline_cf_df.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)
baseline_cf_df_pivot.head()


movieId,1,4,6,7,10,11,15,16,17,18,...,600,601,602,603,604,605,606,607,608,610
userId,,,,,,,,,,,,,,,,,,,,,
1,2.718427,1.183896,1.952224,2.690679,1.311026,1.601204,1.537685,1.0,1.63476,2.920374,...,3.175473,2.015834,1.708640,2.650749,1.894240,2.362099,2.669074,2.619046,3.224402,3.231218
2,1.000000,1.000000,2.135700,1.000000,1.000000,1.486652,1.000000,1.0,1.00000,1.899347,...,2.255102,1.000000,1.859553,1.000000,1.953348,2.034687,1.188273,1.000000,1.978298,2.061377
3,1.220365,1.000000,1.405891,1.000000,1.000000,1.000000,1.000000,1.0,1.00000,1.000000,...,1.080267,1.000000,1.000000,1.000000,1.042914,1.000000,1.000000,1.000000,1.411203,1.000000
5,1.000000,1.000000,1.433410,1.000000,1.000000,1.000000,1.000000,1.0,1.00000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.082075,1.000000,1.000000,1.000000,1.000000,1.000000
6,2.202979,1.000000,1.704203,1.000000,1.000000,1.906122,1.000000,1.0,1.00000,2.027638,...,1.000000,1.000000,1.752253,2.323019,1.443531,1.000000,1.342966,1.834912,2.256911,2.364772


In [16]:
df_movie_features = df_movie_features.unstack("movieId").reset_index()
df_movie_features.columns = ["movieId", "userId", "rating"]
df_movie_features.

In [ ]:
from helpers import MovieLensRatingsDataset
from baseline_cf import *
movielens_dataset = MovieLensRatingsDataset()

baseline_cf_matrix = load_baseline_cf(drop_pickle=True, df=movielens_dataset.full_df)
baseline_cf_matrix.shape

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [24]:
baseline_cf_matrix.shape


(2221, 385)

In [26]:
movielens_dataset = MovieLensRatingsDataset()
movielens_dataset.pivot_df.shape

(385, 2221)